In [1]:
import numpy as np
import pandas as pd

In [2]:
books=pd.read_csv("data/Books.csv")
ratings=pd.read_csv("data/Ratings.csv")
users=pd.read_csv("data/Users.csv")

C:\Users\Pratik Bhilore\AppData\Local\Temp\ipykernel_11664\2000049364.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv("data/Books.csv")


In [3]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [4]:
books.shape

(271360, 8)

In [5]:
users.shape

(278858, 3)

In [6]:
ratings.shape

(1149780, 3)

In [7]:
print("Books",books.columns)
print("Users",users.columns)
print("Ratings",ratings.columns)

Books Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Users Index(['User-ID', 'Location', 'Age'], dtype='object')
Ratings Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


## Popularity Based Recommendation System

In [8]:
ratings_names=ratings.merge(books,on='ISBN')

In [9]:
num_ratings_df=ratings_names.groupby("Book-Title").count()['Book-Rating'].reset_index()
num_ratings_df.rename(columns={"Book-Rating":'num_ratings'},inplace=True)
num_ratings_df.head(10)

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
5,Clifford Visita El Hospital (Clifford El Gran...,1
6,Dark Justice,1
7,Deceived,2
8,Earth Prayers From around the World: 365 Pray...,10
9,Final Fantasy Anthology: Official Strategy Gu...,4


In [10]:
avg_ratings_df=ratings_names.groupby("Book-Title").mean(numeric_only=True)['Book-Rating'].reset_index()
avg_ratings_df.rename(columns={"Book-Rating":'avg_rating'},inplace=True)
avg_ratings_df.head()

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [11]:
popular_df=num_ratings_df.merge(avg_ratings_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [12]:
popular_df=popular_df[popular_df['num_ratings']>250].sort_values('avg_rating',ascending=False).head(50)

In [13]:
popular_df=popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','num_ratings','Image-URL-M','avg_rating']]
popular_df.head()

,Book-Title,Book-Author,num_ratings,Image-URL-M,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,428,http://images.amazon.com/images/P/0439136350.0...,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,387,http://images.amazon.com/images/P/0439139597.0...,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,278,http://images.amazon.com/images/P/0590353403.0...,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,347,http://images.amazon.com/images/P/043935806X.0...,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,556,http://images.amazon.com/images/P/0439064872.0...,5.183453


##  Collaborative Filtering Based Recommendation System

>Selecting Users that have more than 200 ratings for different books
>Selecting books that have more then 50 ratings

In [14]:
ratings_names.groupby('User-ID').count()

,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
User-ID,,,,,,,,,
2,1,1,1,1,1,1,1,1,1
8,17,17,17,17,17,17,17,17,17
9,3,3,3,3,3,3,3,3,3
10,1,1,1,1,1,1,1,1,1
12,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
278846,1,1,1,1,1,1,1,1,1
278849,4,4,4,4,4,4,4,4,4
278851,23,23,23,23,23,23,23,23,23


In [15]:
x=ratings_names.groupby('User-ID').count()['Book-Rating']>200
good_users=x[x].index

In [16]:
filtered_ratings=ratings_names[ratings_names['User-ID'].isin(good_users)]

In [17]:
y=filtered_ratings.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [18]:
final_ratings=filtered_ratings[filtered_ratings['Book-Title'].isin(famous_books)]
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1150,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1163,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
1165,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...
1168,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...
1174,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,http://images.amazon.com/images/P/006440188X.0...,http://images.amazon.com/images/P/006440188X.0...
...,...,...,...,...,...,...,...,...,...,...
1029196,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,http://images.amazon.com/images/P/1400031354.0...,http://images.amazon.com/images/P/1400031354.0...
1029197,275970,1400031362,0,Morality for Beautiful Girls (No.1 Ladies Dete...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031362.0...,http://images.amazon.com/images/P/1400031362.0...,http://images.amazon.com/images/P/1400031362.0...
1029270,275970,1573229725,0,Fingersmith,Sarah Waters,2002,Riverhead Books,http://images.amazon.com/images/P/1573229725.0...,http://images.amazon.com/images/P/1573229725.0...,http://images.amazon.com/images/P/1573229725.0...
1029309,275970,1586210661,9,Me Talk Pretty One Day,David Sedaris,2001,Time Warner Audio Major,http://images.amazon.com/images/P/1586210661.0...,http://images.amazon.com/images/P/1586210661.0...,http://images.amazon.com/images/P/1586210661.0...


In [19]:
pt=final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [20]:
pt.mean(axis=1)

Book-Title
1984                                                                 3.217172
1st to Die: A Novel                                                  2.281457
2nd Chance                                                           2.411504
4 Blondes                                                            0.985714
A Bend in the Road                                                   1.637615
                                                                       ...   
Year of Wonders                                                      3.070175
You Belong To Me                                                     1.545455
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values    1.983333
Zoya                                                                 1.103448
\O\" Is for Outlaw"                                                  2.074468
Length: 706, dtype: float64

In [21]:
 pt.fillna(0,inplace=True)

In [22]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Using ccosine_similarity from sklearn library

In [23]:
from sklearn.metrics.pairwise import  cosine_similarity

In [24]:
similarity_scores=cosine_similarity(pt)
similarity_scores.shape

(706, 706)

In [25]:
np.where(pt.index=='The Notebook')[0][0]

579

## Recommendation Function

In [26]:
def recommend_books(book_name):
    idx=np.where(pt.index==book_name)[0][0]
    distances=similarity_scores[idx]
    similar_books=sorted(list(enumerate(distances)),key=lambda x: x[1],reverse=True)[1:6];

    for i in similar_books:
        print(pt.index[i[0]])
        

## Recommendations

In [27]:
recommend_books('The Notebook')

(24, 0.332547567216117)
A Walk to Remember
(602, 0.2865025090143991)
The Rescue
(362, 0.2550191367154359)
One Door Away from Heaven
(654, 0.2545833265115313)
Toxin
(530, 0.2542410143811226)
The Five People You Meet in Heaven


In [28]:
recommend_books('The Rescue')

(24, 0.44602934936785466)
A Walk to Remember
(579, 0.2865025090143991)
The Notebook
(543, 0.28216083941717207)
The Guardian
(14, 0.278380641352155)
A Man Named Dave: A Story of Triumph and Forgiveness
(518, 0.2719204101694581)
The Dogs of Babel (Today Show Book Club #12)


In [29]:
recommend_books('A Walk to Remember')

(602, 0.44602934936785466)
The Rescue
(355, 0.3373795385806986)
Nights in Rodanthe
(579, 0.332547567216117)
The Notebook
(220, 0.3048194881093923)
Granny Dan
(4, 0.2956261183177974)
A Bend in the Road


## Exporting the data

In [30]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [31]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))